# LTEM Image Simulation Example Notebook
This notebook gives examples of how to simulate and reconstruct LTEM images from a given magnetization. The first section presents an all-in-one function that takes a magnetization vector output file, the various materials and imaging parameters, and returns a TIE reconstructed image set.  

The following sections breaks down the process into the components: magnetization images $\rightarrow$ phase shift $\rightarrow$ LTEM images $\rightarrow$ reconstructed magnetizations.
  
Authors: Arthur McCray, CD Phatak
V1.0, ANL, June 2020

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib notebook
import numpy as np
import sys 
sys.path.append("../PyTIE/")
from TIE_helper import *
from sim_helper import *
from TIE_reconstruct import TIE, SITIE
from comp_phase import mansPhi, linsupPhi

# Simulating from Micromagnetics Output 
reconstruct_ovf() is a single function that begins with a micromagnetics vector output file (.omf or .ovf) and simulates then reconstructs a LTEM through focus series (tfs). The final return is the results dictionary output by TIE(), and the number of images displayed and saved are controlled by simple arguments. There are many parameters that need to be set falling into two primary categories: materials parameters and imaging conditions. 

### Materials Parameters
While the physical dimensions of the sample and voxel size are given in the datafile header, other material parameters are not. Additionally, for practical purposes many samples will be on a substrate or membrane of some sort which we can account for (while assuming it is of uniform thickness). Here we specify: 
* file: The file path. This function can load OVF 2.1/2.0 text files and most binaries as well, though it seems to fail with some binaries output from NanoHub.
* sim: The simulation software used. Outputs from OOMMF and mumax require different scaling. Expects one of the following inputs: 
    - 'oommf': Assumes values given in A/m - scales by mu0
    - 'mumax': Assumes vectors are normalized - scales by Msat
    - 'raw': Does not scale vectors. 
* Msat: The saturation magnetization in gauss. 
* sample_V0: The sample mean inner potential in volts. 
* sample_xip0: The extinction distance for the sample in nm. 
* mem_thk: The thickness of the membrane in nm. 
* mem_xip0: The extinction distance for the membrane in nm. 
* thk_map: 2D array (y,x). Thickness values as factor of total thickness (zscale\*zsize). If a 3D array is given, it will be summed along z axis. Pixels with thickness=0 will not have the phase calculated as a method to speed of computation time; if you want to calculate for all pixels then set thin regions to a small value e.g. 1e-9. Default None -> Uniform thickness as given by datafile, equivalent to array of 1's.

In [ ]:
file = "./example_oommf.omf"
sim = 'oommf'
Msat = 1e4 # gauss
sample_V0 = 20 # V
sample_xip0 = 50 # nm
mem_thk = 50 # nm
mem_xip0 = 1000 # nm
calc_region = None # defaults to full image
thk_map = None # defaults to uniform thickness from the datafile. 

### Imaging conditions
The final reconstruction is dependent on both the imaging conditions as well as the microscope itself. Microscope parameters are contained in a Microscope object (named pscope in the example below), making it easy to compare results in different simulated microscopes.  
Parameters (the default value is given for parameters where it might often be used): 
- defval: The defocus at which to simulate images in nm (in-focus and +/- defval will be used for the tfs)
- theta_x: The sample tilt around the x-axis (degrees). Default 0. 
- theta_y: The sample tilt around the y-axis (degrees). Default 0. 
- add_random: Whether to account for amorphous background in the sample. add_random=1 provides a moderate amount of background as scaled by the electrostatic phase shift. Other float values can be used to increase/decrease the intensity. Default 0. 
- flip: Whether to use a single tfs (False) or calculate a tfs for the sample in both orientations. For flat samples a single tfs is fine, though it's needed for samples with variable thickness (which can be controlled with the thk_map array). Default True. 

In [ ]:
pscope = Microscope(E=200e3, Cs = 200.0e3, theta_c = 0.01e-3, def_spr = 80.0, verbose=True)
defval = 50_000 # nm
theta_x = 0 # degrees
theta_y = 0 # degrees
add_random = 0 # scaling factor
flip=True # Bool

### Calculation Parameters and Run

Finally there are some parameters that control the outputs given. 
* savename: String that will prepend all saved files.
* save: Integer to control the amount of output that is saved; if saving, images and parameters files will be saved to subdirectories of wherever the datafile is located. "/sim_tfs/" will contain the tfs of the unflip and flip simulated images and a params.txt file with basic simulation parameters. "/images/" will contain TIE reconstruction output images. 
    - 0: Saves nothing, still returns results. 
    - 1: Default. Saves simulated images, simulated phase shift, and reconstructed magnetizations as both a color image and x/y components. 
    - 2: Saves simulated images, simulated phase shift, and all reconstruction TIE images. 
* v: Verbosity
    - 0: Suppress all output
    - 1: Default. Standard output with progress on phase shift calculations and displaying resulting image. 
    - 2: Extended output. Prints full datafile header and shows simulated phase shifts and tfs. 


In [ ]:
savename = 'Example'
save=0
v=1

results = reconstruct_ovf(file=file, sim=sim, save=save, savename=savename, v=v, 
                          flip=flip, pscope=pscope, defval=defval, theta_x=theta_x, 
                          theta_y=theta_y, add_random=add_random, thk_map=thk_map,
                          Msat=Msat, sample_V0=sample_V0, sample_xip0=sample_xip0, 
                          mem_thk=mem_thk, mem_xip0=mem_xip0)

### Comparing to the raw magnetization data: 

In [ ]:
from colorwheel import color_im
mag_x, mag_y, mag_z, del_px, zscale = load_ovf(file, sim, Msat, v=0)
show_im(color_im(np.sum(mag_x, axis=0), np.sum(mag_y,axis=0),hsvwheel=True), "Raw magnetization data from file")

# Individual Simulation Components
Here we show how to use the individual components of the simulation. Small constructed vortex states are used to reduce computation time. 

#### Creating a sample magnetization. 

In [ ]:
dim = 64
del_px = 10 # nm/pixel
Bloch_x, Bloch_y, Bloch_z = Bloch(dim, chirality = 'cw', pad = True, ir=0)
show_3D(Bloch_x, Bloch_y, Bloch_z, show_all = True, l=2, a = 50)
show_2D(Bloch_x, Bloch_y, a=50, l=2, title='in-plane magnetization')

## Getting magnetic and electrostatic phase shift with the linear superposition method  
This is the technique used in reconstruct_ovf() to calculate the phase shift of an electron through a given magnetization. It has advantages of being applicable for 3D structures and it's easy to tilt the sample, but it is much slower than the Mansuripur algorithm that will be shown next. 

In [ ]:
del_px = 10 # nm/pix 
b0 = 1e4 # Gauss 
phi0 = 2.07e7 # Gauss*nm^2 
cb = b0/phi0*del_px**2 # 1/px^2
pre_B = 2*np.pi*cb
ephi_L, mphi_L = linsupPhi(mx=Bloch_x.reshape(1,dim,dim),
                           my=Bloch_y.reshape(1,dim,dim),
                           mz=Bloch_z.reshape(1,dim,dim), 
                           Dshp=np.ones(Bloch_z.reshape(1,dim,dim).shape), 
                           pre_B=pre_B)
show_im(mphi_L, "Magnetic phase shift from linear superposition method")

## Calculating the phase shift with the Mansuripur algorithm  
The Mansuripur algorithm is an established technique for calculating phase shifts through 2D magnetizations. It has the advantage of being much faster than the linear superposition method, though not as flexible. For more details seee [this paper by Mansuripur](https://doi.org/10.1063/1.348682).
  
It can be applied easily to island structures using the shape argument which takes a 2D binary array. Here we call std_mansPhi which sets up a basic example using default materials parameters. 

In [ ]:
# Apply mansuripur algorithm with some standard materials parameters. 
ephi, mphi = std_mansPhi(Bloch_x, Bloch_y, del_px=del_px, isl_thk=del_px)
show_im(mphi, title="magnetic phase shift from Mansuripur algorithm")

## Simulating LTEM images from the phase shift
This is where the microscope parameters make the largest difference. 

In [ ]:
# using the linear superposition phase 
pscope = Microscope(E=200e3,Cs = 200.0e3, theta_c = 0.01e-3, def_spr = 80.0)
defval = 50_000
Tphi, im_un, im_in, im_ov = sim_images(mphi=mphi_L, ephi=ephi_L, 
                                       pscope=pscope,
                                       del_px=del_px, 
                                       def_val=defval,
                                       add_random=0.1)
show_sims(Tphi, im_un, im_in, im_ov)

# Full example simulating and reconstructing images from a given magnetization
We simulate the phase shift for a slightly larger region (using the Mansuripur algorithm for speed) and feed the simulated images into the TIE reconstruction routine. Because we specified a uniform sample, we do not need to use a through focus series of the flipped sample. 

In [ ]:
pscope = Microscope(E=200e3,Cs = 200.0e3, theta_c = 0.01e-3, def_spr = 80.0)
dim = 512
del_px = 500/dim

Bloch_x, Bloch_y, Bloch_z = Bloch(dim, chirality = 'cw', pad = True)
ephi, mphi = std_mansPhi(Bloch_x, Bloch_y, del_px = del_px, pscope=pscope)

defval = 100_000
Tphi, im_un, im_in, im_ov = sim_images(mphi = mphi, ephi = ephi, 
                                       pscope = pscope,
                                       del_px = del_px, 
                                       def_val = defval,
                                       add_random = 1)
show_sims(Tphi, im_un, im_in, im_ov)

In [ ]:
ptie = TIE_params(imstack=[im_un, im_in, im_ov], defvals=[defval], flip=False, no_mask=True)
ptie.set_scale(del_px)

i = 0 
dataname = f'Example_Bloch' 
sym = False
qc = False
save = False

results = TIE(i, ptie, pscope, 
                     dataname = dataname, 
                     sym=sym, 
                     qc = qc, 
                     save=save)

#### Comparing the reconstructed phase to the simulated values

In [ ]:
# (total phase shift reconstructed) / (magnetic phase shift simulated)
(np.max(results["phase_m"])-np.min(results["phase_m"]))/(np.max(mphi)-np.min(mphi))

### Single Image TIE (SITIE) Reconstruction 
Because our simulated is uniformly flat and the only source of contrast in our image is magnetic Fresnel contrast, we can actually reconstruct this dataset with just a single image. 
For more information see [this paper by Chess et. al](https://doi.org/10.1016/j.ultramic.2017.02.004). 

In [ ]:
dataname = 'Example_SITIE_Bloch' 
sym = False
qc = False
save = False

results = SITIE(ptie, pscope, 
                     dataname=dataname, 
                     sym=sym, 
                     qc=qc, 
                     save=save,
                     i=0) # for i=1 will try to reconstruct the infocus image and won't work. 

# Other single layer skyrmion magnetization structures
Here we show an alternate method of defining a skyrmion magnetization structure as defined in [a paper by Lillihook et. al](https://doi.org/10.1016/S1386-9477(97)00013-1).

In [ ]:
dim = 64
Bloch_x2, Bloch_y2, Bloch_z2 = Lillihook(dim, Q=1, gamma=3*np.pi/2)
show_3D(Bloch_x2, Bloch_y2, Bloch_z2, show_all=True, l=2, a = 50)

In [ ]:
anti_x, anti_y, anti_z = Lillihook(dim, rad=dim//4, Q = -1, show=False)
show_3D(anti_x, anti_y, anti_z, show_all = False, l=5, a = 25)
show_2D(anti_x, anti_y, l=3, a = 50)

--- End Notebook ---